In [32]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))  

In [33]:
args = {
    "pretrain": '/home/zhulin/pretrain/bert_pretrain_uncased/',
    "model": "./SingleChannelPredictor.pt",
    "dataset": "/mnt/sdd1/data/zhulin/jack/cdatasets.test.5.csv",
    "scores": "/mnt/sdd1/data/zhulin/jack/scores/SingleChannelPredictor.npy",
    # "scores": "/mnt/sdd1/data/zhulin/jack/scores/ExtraTreesClassifier.npy",
}

In [34]:
### load datasets
import numpy as np
import pandas as pd
import datatable as dt

data = dt.fread(args["dataset"], fill=True)
data.head(3)

,index,unique_key,pid,pname,label,channel,cnt
,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,kad29f77ee86ed9827158347befa8998d,−1,2868,explorer.exe,0,RegistryOpen \Registry\Machine\Software\Policies\Microsoft\SQMClient\Windows RegistryOpen \Registry\…,8
1,kad29f77ee86ed9827158347befa8998d,−1,2868,explorer.exe,0,FileIORead C:\Users\Administrator\AppData\Roaming\Microsoft\Windows\Themes\CachedFiles\CachedImage_1…,8
2,kad29f77ee86ed9827158347befa8998d,−1,2868,explorer.exe,0,RegistryQueryValue DisplayVersion RegistryOpen Software\Microsoft\Windows NT\CurrentVersion\Windows …,8


In [35]:
scores = np.load(args["scores"])
data["score"] = scores
data["pred"] = data[:, dt.ifelse(dt.f.score < 0.5, 0, 1)]
data

,index,unique_key,pid,pname,label,channel,cnt,score,pred
,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,kad29f77ee86ed9827158347befa8998d,−1,2868,explorer.exe,0,RegistryOpen \Registry\Machine\Software\Policies\Microsoft\SQMClient\Windows RegistryOpen \Registry\…,8,0.000161278,0
1,kad29f77ee86ed9827158347befa8998d,−1,2868,explorer.exe,0,FileIORead C:\Users\Administrator\AppData\Roaming\Microsoft\Windows\Themes\CachedFiles\CachedImage_1…,8,0.000264826,0
2,kad29f77ee86ed9827158347befa8998d,−1,2868,explorer.exe,0,RegistryQueryValue DisplayVersion RegistryOpen Software\Microsoft\Windows NT\CurrentVersion\Windows …,8,0.000360929,0
3,kad29f77ee86ed9827158347befa8998d,−1,2868,explorer.exe,0,RegistryOpen \REGISTRY\USER\S-1-5-21-2059603325-2600630385-1437625685-500\\SOFTWARE\Microsoft\System…,8,0.00269503,0
4,kad29f77ee86ed9827158347befa8998d,−1,2868,explorer.exe,0,RegistryOpen Control Panel\Desktop RegistryQueryValue PaintDesktopVersion RegistryQuery Software\Mic…,8,0.00218714,0
5,kad29f77ee86ed9827158347befa8998d,−1,2868,explorer.exe,0,RegistryQuery Control Panel\Desktop RegistryOpen AppEvents\Schemes\Apps\.Default\CCSelect\.current R…,8,0.00246246,0
6,kad29f77ee86ed9827158347befa8998d,−1,2868,explorer.exe,0,RegistryOpen \Registry\Machine\SOFTWARE\Microsoft\SystemCertificates\Root\Certificates\2BD63D28D7BCD…,8,0.00168543,0
7,kad29f77ee86ed9827158347befa8998d,−1,2912,ChsIME.exe,0,RegistryOpen \Registry\User\S-1-5-21-2059603325-2600630385-1437625685-500_Classes RegistryQuery Reg…,8,0.00109794,0
8,kad29f77ee86ed9827158347befa8998d,−1,2912,ChsIME.exe,0,RegistryOpen \Registry\User\S-1-5-21-2059603325-2600630385-1437625685-500_Classes RegistryQuery CLSI…,8,0.000521422,0


In [36]:
indexes = data[:, dt.count(), dt.by(dt.f.index)][:, dt.f.index].to_list()[0]
indexes

['k00e3b3952d6cfe18aba4554a034f8e55',
 'k087f42dd5c17b7c42723dfc150a8da42',
 'k0880430c257ce49d7490099d2a8dd01a',
 'k0a47084d98bed02037035d8e3120c241',
 'k0c4502d6655264a9aa420274a0ddeaeb',
 'k168447d837fc71deeee9f6c15e22d4f4',
 'k1de48555aafd904f53e8b19f99658ce8',
 'k209a288c68207d57e0ce6e60ebf60729',
 'k21a563f958b73d453ad91e251b11855c',
 'k2218db42c1b69db72d7432c8d6fcab9d',
 'k25a54e24e9126fba91ccb92143136e9f',
 'k2bbb2d9be1a993a8dfef0dd719c589a0',
 'k2bbff2111232d73a93cd435300d0a07e',
 'k33a7c3fe6c663032798a6780bb21599c',
 'k38393408898e353857a18f481cf15935',
 'k4edfdc708fb7cb3606ca68b6c288f979',
 'k58b70be83f9735f4e626054de966cc94',
 'k5c7fb0927db37372da25f270708103a2',
 'k5e271dbfb5803f600b30f7d9945024fd',
 'k643c8c25fbe8c3cc7576bc8e7bcd8a68',
 'k64497a0fa912f0e190359684de92be2d',
 'k6992dd450b7581d7c2a040d15610a8c5',
 'k6c660f960daac148be75427c712d0134',
 'k6e080aa085293bb9fbdcc9015337d309',
 'k732a229132d455b98038e5a23432385d',
 'k74d9610a72fa9ed105c927e3b1897c5b',
 'k77d0a9541

In [37]:
import collections

vc = []
events = ["ProcessStart", "ProcessEnd", "ThreadStart", "ThreadEnd", "ImageLoad", "FileIOWrite", "FileIORead", "FileIOFileCreate", "FileIORename", "FileIOCreate", "FileIOCleanup", "FileIOClose", "FileIODelete", "FileIOFileDelete", "RegistryCreate", "RegistrySetValue", "RegistryOpen", "RegistryDelete", "RegistrySetInformation", "RegistryQuery", "RegistryQueryValue", "CallStack"]
counter = collections.defaultdict(list)
for index in indexes:
    counter["esindex"].append(index)
    df = data[(dt.f.index == index) & (dt.f.label == 1), :].to_pandas()
    first = df.index[df['pred'] == 1].min()
    if np.isnan(first): first = len(df)

    for event in events:
        ddf = df["channel"].apply(lambda x: x.count(event)).sum()
        counter[event + "All"].append(ddf)
    df = df.iloc[:first]
    for event in events:
        ddf = df["channel"].apply(lambda x: x.count(event)).sum()
        counter[event].append(ddf)
counter
    

defaultdict(list,
            {'esindex': ['k00e3b3952d6cfe18aba4554a034f8e55',
              'k087f42dd5c17b7c42723dfc150a8da42',
              'k0880430c257ce49d7490099d2a8dd01a',
              'k0a47084d98bed02037035d8e3120c241',
              'k0c4502d6655264a9aa420274a0ddeaeb',
              'k168447d837fc71deeee9f6c15e22d4f4',
              'k1de48555aafd904f53e8b19f99658ce8',
              'k209a288c68207d57e0ce6e60ebf60729',
              'k21a563f958b73d453ad91e251b11855c',
              'k2218db42c1b69db72d7432c8d6fcab9d',
              'k25a54e24e9126fba91ccb92143136e9f',
              'k2bbb2d9be1a993a8dfef0dd719c589a0',
              'k2bbff2111232d73a93cd435300d0a07e',
              'k33a7c3fe6c663032798a6780bb21599c',
              'k38393408898e353857a18f481cf15935',
              'k4edfdc708fb7cb3606ca68b6c288f979',
              'k58b70be83f9735f4e626054de966cc94',
              'k5c7fb0927db37372da25f270708103a2',
              'k5e271dbfb5803f600b30f7d9945024fd',
  

In [38]:
df = pd.DataFrame(counter)
df.to_csv("timeliness.csv", index=False, sep="\t")